In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import logging 
logging.basicConfig(level=logging.INFO)
import atlite
from __future__ import annotations

import datetime as dt
import logging
from collections import namedtuple
from operator import itemgetter
from pathlib import Path
from typing import TYPE_CHECKING

import geopandas as gpd
import numpy as np
import pandas as pd
import xarray as xr
from dask import compute, delayed
from dask.array import absolute, arccos, cos, maximum, mod, radians, sin, sqrt, arcsin, arctan2, radians, arctan
from dask.diagnostics import ProgressBar
from numpy import pi
from scipy.sparse import csr_matrix
from atlite.aggregate import aggregate_matrix
from atlite.gis import spdiag
from numpy import pi
import sys
from numpy import logical_and

from atlite import csp as cspm

In [2]:
filename="/groups/EXTREMES/cutouts/europe-2016-era5.nc"
ds_era5_2016 = xr.open_dataset(filename)

In [17]:
ds_era5_2016

<xarray.Dataset> Size: 17GB
Dimensions:           (x: 189, y: 157, time: 8784)
Coordinates:
  * x                 (x) float64 2kB -12.0 -11.75 -11.5 ... 34.5 34.75 35.0
  * y                 (y) float64 1kB 33.0 33.25 33.5 33.75 ... 71.5 71.75 72.0
  * time              (time) datetime64[ns] 70kB 2016-01-01 ... 2016-12-31T23...
    lon               (x) float64 2kB -12.0 -11.75 -11.5 ... 34.5 34.75 35.0
    lat               (y) float64 1kB 33.0 33.25 33.5 33.75 ... 71.5 71.75 72.0
Data variables: (12/13)
    height            (y, x) float32 119kB ...
    wnd100m           (time, y, x) float32 1GB ...
    wnd_azimuth       (time, y, x) float32 1GB ...
    roughness         (time, y, x) float32 1GB ...
    influx_toa        (time, y, x) float32 1GB ...
    influx_direct     (time, y, x) float32 1GB ...
    ...                ...
    albedo            (time, y, x) float32 1GB ...
    solar_altitude    (time, y, x) float64 2GB -1.215 -1.218 ... -0.7083 -0.708
    solar_azimuth     (time, y, x) float64 2GB ...
    temperature       (time, y, x) float64 2GB ...
    soil temperature  (time, y, x) float64 2GB ...
    runoff            (time, y, x) float32 1GB ...
Attributes:
    module:             era5
    prepared_features:  ['influx', 'temperature', 'runoff', 'height', 'wind']
    chunksize_time:     100
    Conventions:        CF-1.6
    history:            2023-02-20 09:11:45 GMT by grib_to_netcdf-2.25.1: /op...

In [3]:
#cutout_path = "europe-2008-06-week-era5_new_wnd10_shear.nc"
cutout_path ="/groups/EXTREMES/cutouts/europe-2016-era5.nc"
 
# read in cutout instead of creating it
cutout = atlite.Cutout(path=cutout_path)

In [4]:
power_era5_2016=cutout.pv(
    panel='CSi', 
    orientation='latitude_optimal',
    tracking=None,
)

INFO:atlite.convert:Convert and aggregate 'pv'.


In [5]:
logger = logging.getLogger(__name__)

In [6]:
def SolarPosition(ds, time_shift="0H"):

    # up to h and dec from [1]

    time_shift = pd.to_timedelta(time_shift)
    #for the models: 
    t = ds.indexes["time"]#.to_datetimeindex()
    t = t + time_shift
    #for the era5
    #t=ds.indexes["time"] + time_shift
    n = xr.DataArray(t.to_julian_date(), coords=ds["time"].coords) - 2451545.0
    hour = (ds["time"] + time_shift).dt.hour
    minute = (ds["time"] + time_shift).dt.minute

    # Operations make new DataArray eager; reconvert to lazy dask arrays
    chunks = ds.chunksizes.get("time", "auto")
    if isinstance(chunks, tuple):
        chunks = chunks[0]
    n = n.chunk(chunks)
    hour = hour.chunk(chunks)
    minute = minute.chunk(chunks)

    L = 280.460 + 0.9856474 * n  # mean longitude (deg)
    g = radians(357.528 + 0.9856003 * n)  # mean anomaly (rad)
    l = radians(L + 1.915 * sin(g) + 0.020 * sin(2 * g))  # ecliptic long. (rad)
    ep = radians(23.439 - 4e-7 * n)  # obliquity of the ecliptic (rad)

    ra = arctan2(cos(ep) * sin(l), cos(l))  # right ascencion (rad)
    lmst = (6.697375 + (hour + minute / 60.0) + 0.0657098242 * n) * 15.0 + ds[
        "lon"
    ]  # local mean sidereal time (deg)
    h = (radians(lmst) - ra + pi) % (2 * pi) - pi  # hour angle (rad)

    dec = arcsin(sin(ep) * sin(l))  # declination (rad)

    # alt and az from [2]
    lat = radians(ds["lat"])
    # Clip before arcsin to prevent values < -1. from rounding errors; can
    # cause NaNs later
    alt = arcsin(
        (sin(dec) * sin(lat) + cos(dec) * cos(lat) * cos(h)).clip(min=-1.0, max=1.0)
    ).rename("altitude")
    alt.attrs["time shift"] = f"{time_shift}"
    alt.attrs["units"] = "rad"

    az = arccos(
        ((sin(dec) * cos(lat) - cos(dec) * sin(lat) * cos(h)) / cos(alt)).clip(
            min=-1.0, max=1.0
        )
    )
    az = az.where(h <= 0, 2 * pi - az).rename("azimuth")
    az.attrs["time shift"] = f"{time_shift}"
    az.attrs["units"] = "rad"

    vars = {da.name: da for da in [alt, az]}
    solar_position = xr.Dataset(vars)

    return solar_position

In [8]:
solar_position_fctn=SolarPosition(ds_era5_2016, time_shift="-30min")
#-30 min because irr data is recorded at 9am for the time between 8am and 9am

In [11]:
solar_position_fctn["altitude"].values

array([[[-1.214567  , -1.21790666, -1.22123657, ..., -1.13262104,
         -1.12911114, -1.12559656],
        [-1.2127748 , -1.21608881, -1.21939275, ..., -1.13135845,
         -1.12786782, -1.12437231],
        [-1.21094051, -1.21422869, -1.21750649, ..., -1.13005886,
         -1.12658772, -1.12311152],
        ...,
        [-0.70149662, -0.70207103, -0.70263887, ..., -0.68590236,
         -0.68517176, -0.68443518],
        [-0.69752988, -0.69809489, -0.69865345, ..., -0.682188  ,
         -0.68146912, -0.68074433],
        [-0.69356037, -0.69411606, -0.69466538, ..., -0.67846941,
         -0.67776217, -0.67704911]],

       [[-1.37930634, -1.3809003 , -1.38242943, ..., -0.91686207,
         -0.91321388, -0.90956493],
        [-1.37538782, -1.3769455 , -1.37843929, ..., -0.91650256,
         -0.91286642, -0.90922944],
        [-1.37145106, -1.37297373, -1.37443347, ..., -0.91611842,
         -0.91249449, -0.90886966],
        ...,
        [-0.72369913, -0.72385168, -0.72399685, ..., -

In [12]:
ds_era5_2016['solar_altitude'].values

array([[[-1.214567  , -1.21790666, -1.22123657, ..., -1.13262104,
         -1.12911114, -1.12559656],
        [-1.2127748 , -1.21608881, -1.21939275, ..., -1.13135845,
         -1.12786782, -1.12437231],
        [-1.21094051, -1.21422869, -1.21750649, ..., -1.13005886,
         -1.12658772, -1.12311152],
        ...,
        [-0.70149662, -0.70207103, -0.70263887, ..., -0.68590236,
         -0.68517176, -0.68443518],
        [-0.69752988, -0.69809489, -0.69865345, ..., -0.682188  ,
         -0.68146912, -0.68074433],
        [-0.69356037, -0.69411606, -0.69466538, ..., -0.67846941,
         -0.67776217, -0.67704911]],

       [[-1.37930634, -1.3809003 , -1.38242943, ..., -0.91686207,
         -0.91321388, -0.90956493],
        [-1.37538782, -1.3769455 , -1.37843929, ..., -0.91650256,
         -0.91286642, -0.90922944],
        [-1.37145106, -1.37297373, -1.37443347, ..., -0.91611842,
         -0.91249449, -0.90886966],
        ...,
        [-0.72369913, -0.72385168, -0.72399685, ..., -

In [14]:
import numpy as np

# Ensure the two datasets are aligned in terms of time and spatial dimensions
aligned_altitude_1, aligned_altitude_2 = xr.align(
    solar_position_fctn['altitude'], 
    ds_era5_2016['solar_altitude'], 
    join="inner"
)

# Calculate the absolute difference between the two datasets
difference = np.abs(aligned_altitude_1 - aligned_altitude_2)

# Check if all values match within a small tolerance (e.g., 1e-6)
tolerance = 1e-6
matches = (difference <= tolerance).all()

# Print the results
print(f"Do the values match within a tolerance of {tolerance}? {matches}")
print(f"Maximum difference: {difference.max().values}")
print(f"Mean difference: {difference.mean().values}")

Do the values match within a tolerance of 1e-06? <xarray.DataArray ()> Size: 1B
dask.array<all-aggregate, shape=(), dtype=bool, chunksize=(), chunktype=numpy.ndarray>
Maximum difference: 0.0
Mean difference: 0.0


In [18]:
import numpy as np

# Ensure the two datasets are aligned in terms of time and spatial dimensions
aligned_azimuth_1, aligned_azimuth_2 = xr.align(
    solar_position_fctn['azimuth'], 
    ds_era5_2016['solar_azimuth'], 
    join="inner"
)

# Calculate the absolute difference between the two datasets
difference = np.abs(aligned_azimuth_1 - aligned_azimuth_2)

# Check if all values match within a small tolerance (e.g., 1e-6)
tolerance = 1e-6
matches = (difference <= tolerance).all()

# Print the results
print(f"Do the values match within a tolerance of {tolerance}? {matches}")
print(f"Maximum difference: {difference.max().values}")
print(f"Mean difference: {difference.mean().values}")

Do the values match within a tolerance of 1e-06? <xarray.DataArray ()> Size: 1B
dask.array<all-aggregate, shape=(), dtype=bool, chunksize=(), chunktype=numpy.ndarray>
Maximum difference: 0.0
Mean difference: 0.0


In [15]:
def get_orientation(name, **params):
    """
    Definitions:
    -`slope` is the angle between ground and panel.
    -`azimuth` is the clockwise angle from North.
        i.e. azimuth = 180 faces exactly South
    """
    if isinstance(name, dict):
        params = name
        name = params.pop("name", "constant")
    return getattr(sys.modules[__name__], f"make_{name}")(**params)


def make_latitude_optimal():
    """
    Returns an optimal tilt angle for the given ``lat``, assuming that the
    panel is facing towards the equator, using a simple method from [1].

    This method only works for latitudes between 0 and 50. For higher
    latitudes, a static 40 degree angle is returned.

    These results should be used with caution, but there is some
    evidence that tilt angle may not be that important [2].

    Function and documentation has been adapted from gsee [3].

    [1] http://www.solarpaneltilt.com/#fixed
    [2] http://dx.doi.org/10.1016/j.solener.2010.12.014
    [3] https://github.com/renewables-ninja/gsee/blob/master/gsee/pv.py

    Parameters
    ----------
    lat : float
        Latitude in degrees.

    """

    def latitude_optimal(lon, lat, solar_position):
        slope = np.empty_like(lat.values)

        below_25 = np.abs(lat.values) <= np.radians(25)
        below_50 = np.abs(lat.values) <= np.radians(50)

        slope[below_25] = 0.87 * np.abs(lat.values[below_25])
        slope[~below_25 & below_50] = 0.76 * np.abs(
            lat.values[~below_25 & below_50]
        ) + np.radians(0.31)
        slope[~below_50] = np.radians(40.0)

        # South orientation for panels on northern hemisphere and vice versa
        azimuth = np.where(lat.values < 0, 0, pi)
        return dict(
            slope=xr.DataArray(slope, coords=lat.coords),
            azimuth=xr.DataArray(azimuth, coords=lat.coords),
        )

    return latitude_optimal


def make_constant(slope, azimuth):
    slope = radians(slope)
    azimuth = radians(azimuth)

    def constant(lon, lat, solar_position):
        return dict(slope=slope, azimuth=azimuth)

    return constant


def make_latitude(azimuth=180):
    azimuth = radians(azimuth)

    def latitude(lon, lat, solar_position):
        return dict(slope=lat, azimuth=azimuth)

    return latitude


def SurfaceOrientation(ds, solar_position, orientation, tracking=None):
    """
    Compute cos(incidence) for slope and panel azimuth.

    References
    ----------
    [1] Sproul, A. B., Derivation of the solar geometric relationships using
    vector analysis, Renewable Energy, 32(7), 1187–1205 (2007).
    [2] Marion, William F., and Aron P. Dobos. Rotation angle for the optimum
    tracking of one-axis trackers. No. NREL/TP-6A20-58891. National Renewable
    Energy Lab.(NREL), Golden, CO (United States), 2013.

    """
    lon = radians(ds["lon"])
    lat = radians(ds["lat"])

    orientation = orientation(lon, lat, solar_position)
    surface_slope = orientation["slope"]
    surface_azimuth = orientation["azimuth"]

    sun_altitude = solar_position["altitude"]
    sun_azimuth = solar_position["azimuth"]

    if tracking is None:
        cosincidence = sin(surface_slope) * cos(sun_altitude) * cos(
            surface_azimuth - sun_azimuth
        ) + cos(surface_slope) * sin(sun_altitude)

    elif tracking == "horizontal":  # horizontal tracking with horizontal axis
        axis_azimuth = orientation[
            "azimuth"
        ]  # here orientation['azimuth'] refers to the azimuth of the tracker axis.
        rotation = arctan(
            (cos(sun_altitude) / sin(sun_altitude)) * sin(sun_azimuth - axis_azimuth)
        )
        surface_slope = abs(rotation)
        surface_azimuth = axis_azimuth + arcsin(
            sin(rotation / sin(surface_slope))
        )  # the 2nd part yields +/-1 and determines if the panel is facing east or west
        cosincidence = cos(surface_slope) * sin(sun_altitude) + sin(
            surface_slope
        ) * cos(sun_altitude) * cos(sun_azimuth - surface_azimuth)

    elif tracking == "tilted_horizontal":  # horizontal tracking with tilted axis'
        axis_tilt = orientation[
            "slope"
        ]  # here orientation['slope'] refers to the tilt of the tracker axis.

        rotation = arctan(
            (cos(sun_altitude) * sin(sun_azimuth - surface_azimuth))
            / (
                cos(sun_altitude) * cos(sun_azimuth - surface_azimuth) * sin(axis_tilt)
                + sin(sun_altitude) * cos(axis_tilt)
            )
        )

        surface_slope = arccos(cos(rotation) * cos(axis_tilt))

        azimuth_difference = sun_azimuth - surface_azimuth
        azimuth_difference = np.where(
            azimuth_difference > pi, 2 * pi - azimuth_difference, azimuth_difference
        )
        azimuth_difference = np.where(
            azimuth_difference < -pi, 2 * pi + azimuth_difference, azimuth_difference
        )
        rotation = np.where(
            logical_and(rotation < 0, azimuth_difference > 0),
            rotation + pi,
            rotation,
        )
        rotation = np.where(
            logical_and(rotation > 0, azimuth_difference < 0),
            rotation - pi,
            rotation,
        )

        cosincidence = cos(rotation) * (
            sin(axis_tilt) * cos(sun_altitude) * cos(sun_azimuth - surface_azimuth)
            + cos(axis_tilt) * sin(sun_altitude)
        ) + sin(rotation) * cos(sun_altitude) * sin(sun_azimuth - surface_azimuth)

    elif tracking == "vertical":  # vertical tracking, surface azimuth = sun_azimuth
        cosincidence = sin(surface_slope) * cos(sun_altitude) + cos(
            surface_slope
        ) * sin(sun_altitude)
    elif tracking == "dual":  # both vertical and horizontal tracking
        cosincidence = np.float64(1.0)
    else:
        assert False, (
            "Values describing tracking system must be None for no tracking,"
            + "'horizontal' for 1-axis horizontal tracking,"
            + "tilted_horizontal' for 1-axis horizontal tracking of tilted panle,"
            + "vertical' for 1-axis vertical tracking, or 'dual' for 2-axis tracking"
        )

    # fixup incidence angle: if the panel is badly oriented and the sun shines
    # on the back of the panel (incidence angle > 90degree), the irradiation
    # would be negative instead of 0; this is prevented here.
    cosincidence = cosincidence.clip(min=0)

    return xr.Dataset(
        {
            "cosincidence": cosincidence,
            "slope": surface_slope,
            "azimuth": surface_azimuth,
        }
    )

In [ ]:
orientation='latitude_optimal'
orientation = get_orientation(orientation)
surface_orientation_fctn=SurfaceOrientation(ds_era5_2016, solar_position_fctn, orientation, tracking=None)
# I do not see a clear way of comparing surface orientation with fctn vs cutout. moving on to next step

In [19]:
trigon_model='simple'
clearsky_model='simple'
tracking=None

In [22]:
# undoing albedo changes applied exclusively for the model case
# SPDX-FileCopyrightText: Contributors to atlite <https://github.com/pypsa/atlite>
#
# SPDX-License-Identifier: MIT

import logging

import numpy as np
from dask.array import cos, fmax, fmin, radians, sin, sqrt

logger = logging.getLogger(__name__)


def DiffuseHorizontalIrrad(ds, solar_position, clearsky_model, influx):
    # Clearsky model from Reindl 1990 to split downward radiation into direct
    # and diffuse contributions. Should switch to more up-to-date model, f.ex.
    # Ridley et al. (2010) http://dx.doi.org/10.1016/j.renene.2009.07.018 ,
    # Lauret et al. (2013):http://dx.doi.org/10.1016/j.renene.2012.01.049

    sinaltitude = sin(solar_position["altitude"])
    influx_toa = ds["influx_toa"]

    if clearsky_model is None:
        clearsky_model = (
            "enhanced" if "temperature" in ds and "humidity" in ds else "simple"
        )

    # Reindl 1990 clearsky model

    k = influx / influx_toa  # clearsky index
    # k.values[k.values > 1.0] = 1.0
    # k = k.rename('clearsky index')

    if clearsky_model == "simple":
        # Simple Reindl model without ambient air temperature and
        # relative humidity
        fraction = (
            ((k > 0.0) & (k <= 0.3))
            * fmin(1.0, 1.020 - 0.254 * k + 0.0123 * sinaltitude)
            + ((k > 0.3) & (k < 0.78))
            * fmin(0.97, fmax(0.1, 1.400 - 1.749 * k + 0.177 * sinaltitude))
            + (k >= 0.78) * fmax(0.1, 0.486 * k - 0.182 * sinaltitude)
        )
    elif clearsky_model == "enhanced":
        # Enhanced Reindl model with ambient air temperature and relative
        # humidity
        T = ds["tas"]
        rh = ds["humidity"]

        fraction = (
            ((k > 0.0) & (k <= 0.3))
            * fmin(
                1.0,
                1.000 - 0.232 * k + 0.0239 * sinaltitude - 0.000682 * T + 0.0195 * rh,
            )
            + ((k > 0.3) & (k < 0.78))
            * fmin(
                0.97,
                fmax(
                    0.1,
                    1.329 - 1.716 * k + 0.267 * sinaltitude - 0.00357 * T + 0.106 * rh,
                ),
            )
            + (k >= 0.78)
            * fmax(0.1, 0.426 * k - 0.256 * sinaltitude + 0.00349 * T + 0.0734 * rh)
        )
    else:
        raise KeyError("`clearsky model` must be chosen from 'simple' and 'enhanced'")

    # Set diffuse fraction to one when the sun isn't up
    # fraction = fraction.where(sinaltitude >= sin(radians(threshold))).fillna(1.0)
    # fraction = fraction.rename('fraction index')

    return (influx * fraction).rename("diffuse horizontal")


def TiltedDiffuseIrrad(ds, solar_position, surface_orientation, direct, diffuse):
    # Hay-Davies Model

    sinaltitude = sin(solar_position["altitude"])
    influx_toa = ds["influx_toa"]

    cosincidence = surface_orientation["cosincidence"]
    surface_slope = surface_orientation["slope"]

    influx = direct + diffuse

    with np.errstate(divide="ignore", invalid="ignore"):
        # brightening factor
        f = sqrt(direct / influx).fillna(0.0)

        # anisotropy factor
        A = direct / influx_toa

    # geometric factor
    R_b = cosincidence / sinaltitude

    diffuse_t = (
        (1.0 - A)
        * ((1 + cos(surface_slope)) / 2.0)
        * (1.0 + f * sin(surface_slope / 2.0) ** 3)
        + A * R_b
    ) * diffuse

    # fixup: clip all negative values (unclear why it gets negative)
    # note: REatlas does not do the fixup
    if logger.isEnabledFor(logging.WARNING):
        if ((diffuse_t < 0.0) & (sinaltitude > sin(radians(1.0)))).any():
            logger.warning(
                "diffuse_t exhibits negative values above altitude threshold."
            )

    with np.errstate(invalid="ignore"):
        diffuse_t = diffuse_t.clip(min=0).fillna(0)

    return diffuse_t.rename("diffuse tilted")


def TiltedDirectIrrad(solar_position, surface_orientation, direct):
    sinaltitude = sin(solar_position["altitude"])
    cosincidence = surface_orientation["cosincidence"]

    # geometric factor
    R_b = cosincidence / sinaltitude

    return (R_b * direct).rename("direct tilted")


def _albedo(ds, influx):
    if "albedo" in ds:
        albedo = ds["albedo"]
    elif "outflux" in ds:
        albedo = (ds["outflux"] / influx.where(influx != 0)).fillna(0).clip(max=1)
    else:
        raise AssertionError(
            "Need either albedo or outflux as a variable in the dataset. "
            "Check your cutout and dataset module."
        )

    return albedo


def TiltedGroundIrrad(ds, solar_position, surface_orientation, influx):
    surface_slope = surface_orientation["slope"]
    ground_t = influx * _albedo(ds, influx) * (1.0 - cos(surface_slope)) / 2.0
    return ground_t.rename("ground tilted")


def TiltedIrradiation(
    ds,
    solar_position,
    surface_orientation,
    trigon_model,
    clearsky_model,
    tracking=0,
    altitude_threshold=1.0,
    irradiation="total",
):
    """
    Calculate the irradiation on a tilted surface.

    Parameters
    ----------
    ds : xarray.Dataset
        Cutout data used for calculating the irradiation on a tilted surface.
    solar_position : xarray.Dataset
        Solar position calculated using atlite.pv.SolarPosition,
        containing a solar 'altitude' (in rad, 0 to pi/2) for the 'ds' dataset.
    surface_orientation : xarray.Dataset
        Surface orientation calculated using atlite.orientation.SurfaceOrientation.
    trigon_model : str
        Type of trigonometry model. Defaults to 'simple'if used via `convert_irradiation`.
    clearsky_model : str or None
        Either the 'simple' or the 'enhanced' Reindl clearsky
        model. The default choice of None will choose dependending on
        data availability, since the 'enhanced' model also
        incorporates ambient air temperature and relative humidity.
        NOTE: this option is only used if the used climate dataset
        doesn't provide direct and diffuse irradiation separately!
    altitude_threshold : float
        Threshold for solar altitude in degrees. Values in range (0, altitude_threshold]
        will be set to zero. Default value equals 1.0 degrees.
    irradiation : str
        The irradiation quantity to be returned. Defaults to "total" for total
        combined irradiation. Other options include "direct" for direct irradiation,
        "diffuse" for diffuse irradation, and "ground" for irradiation reflected
        by the ground via albedo. NOTE: "ground" irradiation is not calculated
        by all `trigon_model` options in the `convert_irradiation` method,
        so use with caution!

    Returns
    -------
    result : xarray.DataArray
        The desired irradiation quantity on the tilted surface.

    """
    influx_toa = ds["influx_toa"]

    def clip(influx, influx_max):
        # use .data in clip due to dask-xarray incompatibilities
        return influx.clip(min=0, max=influx_max.transpose(*influx.dims).data)

    if "influx" in ds:
        influx = clip(ds["influx"], influx_toa)
        diffuse = DiffuseHorizontalIrrad(ds, solar_position, clearsky_model, influx)
        direct = influx - diffuse
    elif "influx_direct" in ds and "influx_diffuse" in ds:
        direct = clip(ds["influx_direct"], influx_toa)
        diffuse = clip(ds["influx_diffuse"], influx_toa - direct)
    else:
        raise AssertionError(
            "Need either influx or influx_direct and influx_diffuse in the "
            "dataset. Check your cutout and dataset module."
        )
    if trigon_model == "simple":
        k = surface_orientation["cosincidence"] / sin(solar_position["altitude"])
        if tracking != "dual":
            cos_surface_slope = cos(surface_orientation["slope"])
        elif tracking == "dual":
            cos_surface_slope = sin(solar_position["altitude"])

        influx = direct + diffuse
        direct_t = k * direct
        diffuse_t = (1.0 + cos_surface_slope) / 2.0 * diffuse
        ground_t = _albedo(ds, influx) * influx * ((1.0 - cos_surface_slope) / 2.0)

        total_t = direct_t.fillna(0.0) + diffuse_t.fillna(0.0) + ground_t.fillna(0.0)
    else:
        diffuse_t = TiltedDiffuseIrrad(
            ds, solar_position, surface_orientation, direct, diffuse
        )
        direct_t = TiltedDirectIrrad(solar_position, surface_orientation, direct)
        ground_t = TiltedGroundIrrad(
            ds, solar_position, surface_orientation, direct + diffuse
        )

        total_t = direct_t + diffuse_t + ground_t

    if irradiation == "total":
        result = total_t.rename("total tilted")
    elif irradiation == "direct":
        result = direct_t.rename("direct tilted")
    elif irradiation == "diffuse":
        result = diffuse_t.rename("diffuse tilted")
    elif irradiation == "ground":
        result = ground_t.rename("ground tilted")

    # The solar_position algorithms have a high error for small solar altitude
    # values, leading to big overall errors from the 1/sinaltitude factor.
    # => Suppress irradiation below solar altitudes of 1 deg.

    cap_alt = solar_position["altitude"] < radians(altitude_threshold)
    result = result.where(~(cap_alt | (direct + diffuse <= 0.01)), 0)
    result.attrs["units"] = "W m**-2"

    return result

In [24]:
irradiation_fctn=TiltedIrradiation(
    ds_era5_2016,
    solar_position_fctn,
    surface_orientation_fctn,
    trigon_model,
    clearsky_model,
    tracking=0,
    altitude_threshold=1.0,
    irradiation="total",
)

In [28]:
# SPDX-FileCopyrightText: Contributors to atlite <https://github.com/pypsa/atlite>
#
# SPDX-License-Identifier: MIT

import numpy as np

# Huld model was copied from gsee -- global solar energy estimator
# by Stefan Pfenninger
# https://github.com/renewables-ninja/gsee/blob/master/gsee/pv.py


def _power_huld(irradiance, t_amb, pc):
    """
    AC power per capacity predicted by Huld model, based on W/m2 irradiance.

    Maximum power point tracking is assumed.

    [1] Huld, T. et al., 2010. Mapping the performance of PV modules,
    effects of module type and data averaging. Solar Energy, 84(2),
    p.324-338. DOI: 10.1016/j.solener.2009.12.002
    """
    # normalized module temperature
    T_ = (pc["c_temp_amb"] * t_amb + pc["c_temp_irrad"] * irradiance) - pc["r_tmod"]

    # normalized irradiance
    G_ = irradiance / pc["r_irradiance"]

    log_G_ = np.log(G_.where(G_ > 0))
    # NB: np.log without base implies base e or ln
    eff = (
        1
        + pc["k_1"] * log_G_
        + pc["k_2"] * (log_G_) ** 2
        + T_ * (pc["k_3"] + pc["k_4"] * log_G_ + pc["k_5"] * log_G_**2)
        + pc["k_6"] * (T_**2)
    )

    eff = eff.fillna(0.0).clip(min=0)

    da = G_ * eff * pc.get("inverter_efficiency", 1.0)
    da.attrs["units"] = "kWh/kWp"
    da = da.rename("specific generation")

    return da


def _power_bofinger(irradiance, t_amb, pc):
    """
    AC power per capacity predicted by bofinger model, based on W/m2
    irradiance.

    Maximum power point tracking is assumed.

    [2] Hans Beyer, Gerd Heilscher and Stefan Bofinger, 2004. A robust
    model for the MPP performance of different types of PV-modules
    applied for the performance check of grid connected systems.
    """
    fraction = (pc["NOCT"] - pc["Tamb"]) / pc["Intc"]

    eta_ref = (
        pc["A"]
        + pc["B"] * irradiance
        + pc["C"] * np.log(irradiance.where(irradiance != 0))
    )
    eta = (
        eta_ref
        * (1.0 + pc["D"] * (fraction * irradiance + (t_amb - pc["Tstd"])))
        / (1.0 + pc["D"] * fraction / pc["ta"] * eta_ref * irradiance)
    ).fillna(0)

    capacity = (pc["A"] + pc["B"] * 1000.0 + pc["C"] * np.log(1000.0)) * 1e3
    power = irradiance * eta * (pc.get("inverter_efficiency", 1.0) / capacity)
    power = power.where(irradiance >= pc["threshold"], 0)
    return power.rename("AC power")


def SolarPanelModel(ds, irradiance, pc):
    model = pc.get("model", "huld")

    if model == "huld":
        return _power_huld(irradiance, ds["temperature"], pc)
    elif model == "bofinger":
        return _power_bofinger(irradiance, ds["temperature"], pc)
    else:
        AssertionError(f"Unknown panel model: {model}")

In [29]:
panel = {
    "model": "huld",  # Model type
    "name": "CSi",  # Panel name
    "source": "Huld 2010",  # Source of the model

    # Used for calculating capacity per m2
    "efficiency": 0.1,  # Efficiency of the panel

    # Panel temperature coefficients
    "c_temp_amb": 1,  # Panel temperature coefficient of ambient temperature
    "c_temp_irrad": 0.035,  # Panel temperature coefficient of irradiance (K / (W/m2))

    # Reference conditions
    "r_tamb": 293,  # Reference ambient temperature (20 degC in Kelvin)
    "r_tmod": 298,  # Reference module temperature (25 degC in Kelvin)
    "r_irradiance": 1000,  # Reference irradiance (W/m^2)

    # Fitting parameters
    "k_1": -0.017162,
    "k_2": -0.040289,
    "k_3": -0.004681,
    "k_4": 0.000148,
    "k_5": 0.000169,
    "k_6": 0.000005,

    # Inverter efficiency
    "inverter_efficiency": 0.9
}

In [30]:
solar_panel_fctn = SolarPanelModel(ds_era5_2016,irradiation_fctn, panel)

In [31]:
aggregated_generation_fctn=solar_panel_fctn.sum(dim="time")

In [ ]:
aggregated_generation_fctn

In [34]:
aggregated_generation_fctn.values

array([[1665.117087  , 1670.82985761, 1675.62453298, ..., 1733.68983396,
        1724.15118419, 1713.5150573 ],
       [1654.5333218 , 1660.26905172, 1667.04610104, ..., 1732.17706531,
        1724.57313397, 1720.71018939],
       [1646.86677728, 1653.54548917, 1660.31920306, ..., 1732.02486402,
        1725.92714883, 1722.08074039],
       ...,
       [ 668.541382  ,  663.58420748,  660.24277091, ...,  717.41877686,
         716.21786177,  715.0961002 ],
       [ 672.07585724,  666.2621657 ,  661.25752118, ...,  705.85857593,
         704.28351569,  702.56793633],
       [ 673.47312207,  667.58092958,  662.66832355, ...,  691.90428341,
         690.56089424,  688.89504447]])

In [35]:
power_era5_2016.values

array([[1665.117087  , 1670.82985761, 1675.62453298, ..., 1733.68983396,
        1724.15118419, 1713.5150573 ],
       [1654.5333218 , 1660.26905172, 1667.04610104, ..., 1732.17706531,
        1724.57313397, 1720.71018939],
       [1646.86677728, 1653.54548917, 1660.31920306, ..., 1732.02486402,
        1725.92714883, 1722.08074039],
       ...,
       [ 668.541382  ,  663.58420748,  660.24277091, ...,  717.41877686,
         716.21786177,  715.0961002 ],
       [ 672.07585724,  666.2621657 ,  661.25752118, ...,  705.85857593,
         704.28351569,  702.56793633],
       [ 673.47312207,  667.58092958,  662.66832355, ...,  691.90428341,
         690.56089424,  688.89504447]])

In [42]:
aggregated_generation_fctn[100:200].values

array([[973.09276915, 972.87205222, 972.20841409, ..., 881.44710599,
        885.1639935 , 887.99152864],
       [961.43100413, 961.54549297, 961.1808881 , ..., 872.52346725,
        877.41962459, 883.08042139],
       [947.21469277, 947.58840844, 948.17270899, ..., 865.27246557,
        869.36408497, 874.35117172],
       ...,
       [668.541382  , 663.58420748, 660.24277091, ..., 717.41877686,
        716.21786177, 715.0961002 ],
       [672.07585724, 666.2621657 , 661.25752118, ..., 705.85857593,
        704.28351569, 702.56793633],
       [673.47312207, 667.58092958, 662.66832355, ..., 691.90428341,
        690.56089424, 688.89504447]])

In [43]:
power_era5_2016[100:200].values

array([[973.09276915, 972.87205222, 972.20841409, ..., 881.44710599,
        885.1639935 , 887.99152864],
       [961.43100413, 961.54549297, 961.1808881 , ..., 872.52346725,
        877.41962459, 883.08042139],
       [947.21469277, 947.58840844, 948.17270899, ..., 865.27246557,
        869.36408497, 874.35117172],
       ...,
       [668.541382  , 663.58420748, 660.24277091, ..., 717.41877686,
        716.21786177, 715.0961002 ],
       [672.07585724, 666.2621657 , 661.25752118, ..., 705.85857593,
        704.28351569, 702.56793633],
       [673.47312207, 667.58092958, 662.66832355, ..., 691.90428341,
        690.56089424, 688.89504447]])

In [44]:
import numpy as np
import xarray as xr

# Ensure the two datasets are aligned in terms of spatial dimensions
aligned_gen_fctn, aligned_power_era5 = xr.align(
    aggregated_generation_fctn, 
    power_era5_2016, 
    join="inner"
)

# Define a tolerance for comparison
tolerance = 1e-6

# Compare the values with the tolerance
differences = np.abs(aligned_gen_fctn - aligned_power_era5) > tolerance

# Count the number of differing values
num_differences = differences.sum().compute()  # Use .compute() to evaluate the Dask array

# Print the results
print(f"Total number of values in cutout: {aligned_power_era5.size}")
print(f"Total number of values in fctn: {aligned_gen_fctn.size}")
print(f"Number of differing values: {num_differences.values}")
print(f"Percentage of differing values: {100 * num_differences.values / aligned_power_era5.size:.2f}%")

Total number of values in cutout: 29673
Total number of values in fctn: 29673
Number of differing values: 0
Percentage of differing values: 0.00%
